In [1]:
import io
import os
import shutil
import json
import lz4.frame
from time import perf_counter
from multiprocessing import Pool
from glob import glob


import numpy as np
import matplotlib.pyplot as plt


from CompressArchive import CompressArchive
from utils import *
import massPy as mp


#import massPy_dev.massPy as mp

In [4]:
main_path = 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data'
out_dir = os.path.join(main_path, 'compressed_data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

data_dirs = os.listdir(main_path)
data_dirs = [os.path.join(main_path, d) for d in data_dirs]
N = -3
dir = data_dirs[N]

outfiles_paths = []
for file in os.listdir(dir):
    if not file.endswith('.json'):
        outfiles_paths.append(os.path.join(dir, file))

for file in outfiles_paths:
    os.remove(file)

file_path = os.path.join(dir, os.listdir(dir)[0])
dir, file_path

('C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\qzk1k30.05_K30.05_qkbt0_z0.022_xi1_LX256_counter0',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\qzk1k30.05_K30.05_qkbt0_z0.022_xi1_LX256_counter0\\frame10000.json')

In [7]:
files = glob(os.path.join(dir, 'frame*'))
#for file in glob(os.path.join(dir, 'frame*')):
 #   print(file)

In [16]:
os.path.dirname(files[0])

'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\qzk1k30.05_K30.05_qkbt0_z0.022_xi1_LX256_counter0'

In [13]:
parent = os.path.join(files[0], os.pardir) 

In [14]:
parent

'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\qzk1k30.05_K30.05_qkbt0_z0.022_xi1_LX256_counter0\\frame10000.json\\..'

In [5]:
data_dirs

['C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\compressed_data',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\ns1024_06',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\ns1024_06_npz',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\ns512_06',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\ns512_19n16',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\ns512_19n16_compressed',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\ns512_19n3',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\ns512_205',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\ns512_205_compressed',
 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\

In [10]:
ca = CompressArchive(dir, dtype_out = 'float32', delete_archive_if_successful = True)
conversion_kwargs = {'compress': True, 'exclude_keys': ['ff'], 'calc_velocities': True, 'overwrite_existing_npz_files': True, 'verbose': 1}
ca.convert_archive_to_npz(**conversion_kwargs)

Archive processed in 65.35 seconds with 0 failed conversions

Estimated size reduction for entire archive (181 frames):
Uncompressed archive size: 1191.86 MB
Compressed archive size: 199.42 MB
Compression ratio: 5.98x

Time to open json frame: 0.18 seconds
Time to open npz frame: 0.01 seconds
Speedup: 17.05x
Archive C:\Users\Simon Andersen\Dokumenter\Uni\Speciale\Hyperuniformity\nematic_data\qzk1k30.05_K30.05_qkbt0_z0.1_xi1_LX256_counter0 deleted


In [3]:
def find_missing_frames(archive_path):

    ar = mp.archive.loadarchive(archive_path)

    dir_list = os.listdir(archive_path)
    frame_list = []

    for item in dir_list:
        if item.startswith("frame"):
            frame_num = int(item.split('.')[0].split('frame')[-1])
            frame_list.append(frame_num)

    if len(frame_list) == ar.num_frames:
        return np.arange(ar.nstart, ar.nsteps + 1, ar.ninfo)
    else:
        frame_list.sort()
        return frame_list

In [4]:
dir

'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\ns1024_06'

In [5]:
def create_npz_folder(archive_path, output_folder = None, check_for_missing_frames = False, compress = True, \
                      dtype_out= 'float32', exclude_keys=[], verbose = 1):
    """
    verbose = 0: no output
    verbose = 1: print time to process entire archive
    verbose = 2: print time to process each frame
    """
    # Create the output folder if it does not exist

    output_folder = archive_path + '_npz' if output_folder is None else output_folder

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # copy parameters.json to output folder
    parameters_path = os.path.join(archive_path, 'parameters.json')
    shutil.copy(parameters_path, output_folder)

    # Load the archive and get the list of frames
    ar = mp.archive.loadarchive(archive_path)
    frame_list = find_missing_frames(archive_path) if check_for_missing_frames else np.arange(ar.nstart, ar.nsteps + 1, ar.ninfo)

    # initialize failed conversions list
    failed_conversions = []

    if verbose > 0:
        start = time.perf_counter()

    for i, frame in enumerate(frame_list):
        frame_input_path = os.path.join(archive_path, f'frame{frame}.json')
        frame_output_path = os.path.join(output_folder, f'frame{frame}.npz')
        try:
            if verbose == 2:
                start_frame = time.perf_counter()
            convert_json_to_npz(frame_input_path, frame_output_path, compress = compress, dtype_out = dtype_out, exclude_keys = exclude_keys)
            if verbose == 2:
                print(f'Frame {frame} processed in {time.perf_counter() - start_frame:.2f} seconds')
        except:
            print(f'Error processing frame {frame}. Skipping...')
            failed_conversions.append(frame)

    if verbose > 0:
        print(f'Archive processed in {time.perf_counter() - start:.2f} seconds with {len(failed_conversions)} failed conversions')
        if len(failed_conversions) > 0:
            print(f'Frames for which conversion to npz failed: {failed_conversions}')
        print('\nEstimated (from first frame) size reduction of archive: ')

        frame_input_path = os.path.join(archive_path, f'frame{frame_list[0]}.json')
        frame_output_path = os.path.join(output_folder, f'frame{frame_list[0]}.npz')
        input_size, output_size, ratio = estimate_size_reduction(frame_input_path, frame_output_path, Nframes = len(frame_list))
    return


In [6]:
create_npz_folder(dir, compress = True, check_for_missing_frames=True, verbose = 1)

Archive processed in 17.18 seconds with 0 failed conversions

Estimated (from first frame) size reduction of archive: 
Uncompressed archive size: 311.12 MB
Compressed archive size: 100.17 MB
Compression ratio: 3.11x



In [ ]:
paths = ['wet_z_phasetwist05/']

for path in paths:
    names = glob.glob(f'{path}/*/*')

nnames = len(names)
print(nnames)
#sys.exit()
ntasks = min(nnames, os.cpu_count())
csize = 10

with Pool(ntasks) as p:
        p.map(zip_file, names, chunksize=csize)

In [37]:
def convert_json_to_npz(frame_number, archive_path, output_folder, compress = True, dtype_out = 'float64', exclude_keys=[]):

    frame_input_path = os.path.join(archive_path, f'frame{frame_number}.json')
    frame_output_path = os.path.join(output_folder, f'frame{frame_number}.npz')

    with open(frame_input_path, 'r') as f:
        data = json.load(f)

    keys = list(data['data'].keys())
    arr_dict = {}
    arr_dict = {key: np.array(data['data'][key]['value'],dtype=dtype_out) for key in keys if key not in exclude_keys}


    if compress:
        np.savez_compressed(frame_output_path, **arr_dict)
    else:
        np.savez(frame_output_path, **arr_dict)
    print(f'Frame {frame_number} processed')
    return


In [32]:
def create_npz_folder2(archive_path, output_folder = None, check_for_missing_frames = False, compress = True, \
                      dtype_out= 'float32', exclude_keys=[], verbose = 1):
    """
    verbose = 0: no output
    verbose = 1: print time to process entire archive
    verbose = 2: print time to process each frame
    """
    # Create the output folder if it does not exist

    output_folder = archive_path + '_npz' if output_folder is None else output_folder

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # copy parameters.json to output folder
    parameters_path = os.path.join(archive_path, 'parameters.json')
    shutil.copy(parameters_path, output_folder)

    # Load the archive and get the list of frames
    ar = mp.archive.loadarchive(archive_path)
    frame_list = find_missing_frames(archive_path) if check_for_missing_frames else np.arange(ar.nstart, ar.nsteps + 1, ar.ninfo)

    # initialize failed conversions list
    failed_conversions = []

    if verbose > 0:
        start = time.perf_counter()

    
  
    ntasks = min(len(frame_list), os.cpu_count())
    csize = 10

    conversion_func = lambda frame_number: convert_json_to_npz(frame_number, \
                                         archive_path=archive_path, output_folder=output_folder, compress = compress, dtype_out = dtype_out, exclude_keys = exclude_keys)

    print(frame_list)
    print(ntasks)
    print(csize)
    with Pool(ntasks) as p:
            p.map_async(conversion_func, frame_list, chunksize=csize)

    if 0:
        for i, frame in enumerate(frame_list):
            frame_input_path = os.path.join(archive_path, f'frame{frame}.json')
            frame_output_path = os.path.join(output_folder, f'frame{frame}.npz')
            try:
                if verbose == 2:
                    start_frame = time.perf_counter()
                convert_json_to_npz(frame_input_path, frame_output_path, compress = compress, dtype_out = dtype_out, exclude_keys = exclude_keys)
                if verbose == 2:
                    print(f'Frame {frame} processed in {time.perf_counter() - start_frame:.2f} seconds')
            except:
                print(f'Error processing frame {frame}. Skipping...')
                failed_conversions.append(frame)

    if verbose > 0:
        print(f'Archive processed in {time.perf_counter() - start:.2f} seconds')
        if len(failed_conversions) > 0:
            print(f'Frames for which conversion to npz failed: {failed_conversions}')
        print('\nEstimated (from first frame) size reduction of archive: ')

        frame_input_path = os.path.join(archive_path, f'frame{frame_list[0]}.json')
        frame_output_path = os.path.join(output_folder, f'frame{frame_list[0]}.npz')
        input_size, output_size, ratio = estimate_size_reduction(frame_input_path, frame_output_path, Nframes = len(frame_list))
    return

In [35]:
list(np.arange(0, 10, 1) )

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [41]:
frame_list= list(np.arange(10000, 14000, 500))

ntasks = min(len(frame_list), 2)
csize = 10

archive_path = dir  
output_folder = dir + '_npz'
compress = True
dtype_out= 'float32'
exclude_keys=[]
conversion_func = lambda frame_number: convert_json_to_npz(frame_number, \
                                      archive_path=archive_path, output_folder=output_folder, compress = compress, dtype_out = dtype_out, exclude_keys = exclude_keys)


conversion_func(frame_list[0])

with Pool(ntasks) as p:
        p.map(conversion_func, frame_list, chunksize=csize)

Frame 10000 processed


PicklingError: Can't pickle <function <lambda> at 0x00000292B6E8AE50>: attribute lookup <lambda> on __main__ failed

In [43]:
# print "[0, 1, 4,..., 81]"
def f(x):
    return x*x
with Pool(5) as pool:
    print(pool.map(f, range(10)))   


In [33]:
create_npz_folder2(dir, compress = True, check_for_missing_frames=True, verbose = 1)

[ 10000  10500  11000  11500  12000  12500  13000  13500  14000  14500
  15000  15500  16000  16500  17000  17500  18000  18500  19000  19500
  20000  20500  21000  21500  22000  22500  23000  23500  24000  24500
  25000  25500  26000  26500  27000  27500  28000  28500  29000  29500
  30000  30500  31000  31500  32000  32500  33000  33500  34000  34500
  35000  35500  36000  36500  37000  37500  38000  38500  39000  39500
  40000  40500  41000  41500  42000  42500  43000  43500  44000  44500
  45000  45500  46000  46500  47000  47500  48000  48500  49000  49500
  50000  50500  51000  51500  52000  52500  53000  53500  54000  54500
  55000  55500  56000  56500  57000  57500  58000  58500  59000  59500
  60000  60500  61000  61500  62000  62500  63000  63500  64000  64500
  65000  65500  66000  66500  67000  67500  68000  68500  69000  69500
  70000  70500  71000  71500  72000  72500  73000  73500  74000  74500
  75000  75500  76000  76500  77000  77500  78000  78500  79000  79500
  8000

FileNotFoundError: [WinError 2] Den angivne fil blev ikke fundet: 'C:\\Users\\Simon Andersen\\Dokumenter\\Uni\\Speciale\\Hyperuniformity\\nematic_data\\qzk1k30.05_K30.05_qkbt0_z0.022_xi1_LX256_counter0_npz\\frame10000.npz'

In [4]:
ar = mp.archive.loadarchive(ca.output_dir)
frame = ar._read_frame(0)
defects = get_defect_list(ar, Nframes=1)
def_arr = get_defect_arr_from_frame(defects[0])

In [7]:
# Time to open array and get defects using current method
t1 = perf_counter()
ar = mp.archive.loadarchive(dir)
frame = ar._read_frame(0)

Qxx_dat = frame.QQxx.reshape(ar.LX, ar.LY)
Qyx_dat = frame.QQyx.reshape(ar.LX, ar.LY)
#defect_list = get_defect_list(ar, ar.LX, ar.LY, Nframes=2, archive_path=dir)
t2 = perf_counter()
print('Time to open array and get defects using current method: ', (t2-t1)/(len(os.listdir(dir)) -1))

Time to open array and get defects using current method:  6.95262305


In [17]:
# Time to open array using load json
t1 = perf_counter()
with open(file_path, 'r') as f:
    data = json.load(f)
arr_dict = unpack_arrays(data)
arr_dict32 = unpack_arrays(data, dtype_out='float32')
t2 = perf_counter()
print('Time to open array using load json: ', t2-t1)

Time to open array using load json:  6.14445180000007


In [20]:
# Time to convert and save to numpy array
t1 = perf_counter()
npz_file_res = os.path.join(dir, os.path.join(out_dir,'data.npz'))
with open(file_path, 'r') as f:
    data = json.load(f)
arr_dict = unpack_arrays(data, dtype_out = 'float32')
del arr_dict['ff']
np.savez_compressed(npz_file_res, **arr_dict)
t2 = perf_counter()

print('Time to convert and save to numpy array, excluding ff: ', t2-t1)
print_size(file_path, npz_file_res)

# Time to open npz file and unpack arrays
t1 = perf_counter()
npz = np.load(npz_file_res, allow_pickle=True)
LX = ar.LX

Qxx_dat = npz['QQxx'].reshape(LX, LX)
Qyx_dat = npz['QQyx'].reshape(LX, LX)
# Get defects
defects = mp.nematic.nematicPy.get_defects(Qxx_dat, Qyx_dat, LX, LX)


t2 = perf_counter()
print('Time to open npz file and get defects, excluding ff: ', t2-t1)

Time to convert and save to numpy array, excluding ff:  7.462377200000105
Input file size: 155.5849323272705 MB
Output file size: 22.298182487487793 MB
Compression ratio: 6.98x
Time to open npz file and get defects, excluding ff:  1.4298758999998427


In [19]:
# Time to convert and save to numpy array
t1 = perf_counter()
npz_file = os.path.join(dir, os.path.join(out_dir,'data.npz'))
with open(file_path, 'r') as f:
    data = json.load(f)
arr_dict = unpack_arrays(data, dtype_out = 'float32')
np.savez_compressed(npz_file, **arr_dict)
t2 = perf_counter()

print('Time to convert and save to numpy array: ', t2-t1)
print_size(file_path, npz_file)

# Time to open npz file and unpack arrays
t1 = perf_counter()
npz = np.load(npz_file, allow_pickle=True)
LX = ar.LX

Qxx_dat = npz['QQxx'].reshape(LX, LX)
Qyx_dat = npz['QQyx'].reshape(LX, LX)
# Get defects
defects = mp.nematic.nematicPy.get_defects(Qxx_dat, Qyx_dat, LX, LX)


t2 = perf_counter()
print('Time to open npz file and get defects: ', t2-t1)

Time to convert and save to numpy array:  8.52731899999992
Input file size: 155.5849323272705 MB
Output file size: 50.06804656982422 MB
Compression ratio: 3.11x
Time to open npz file and get defects:  1.2702217999999448


In [15]:
# Time to compress json file using lz4
t1 = perf_counter()
output_file = file_path + '.lz4'
compress_file(file_path, output_file)
t2 = perf_counter()
print('Time to compress json file using lz4: ', t2-t1)
print_size(file_path, output_file)

# Time to read lz4 file, decompress and load arrays
t1 = perf_counter()
json_dict = decompress_and_convert(output_file, out_format='json')
arr_dict = unpack_arrays(json_dict)
t2 = perf_counter()
print('Time to compress json file using lz4: ', t2-t1)

Time to compress json file using lz4:  0.7508942000000047
Input file size: 155.5849323272705 MB
Output file size: 89.09958457946777 MB
Compression ratio: 1.75x
Time to compress json file using lz4:  4.974672699999985


In [16]:
# Time to compress npz file using lz4
t1 = perf_counter()
output_file = npz_file + '.lz4'
compress_file(npz_file, output_file)
t2 = perf_counter()
print('Time to compress npz file using lz4: ', t2-t1)
print_size(npz_file, output_file)

# Time to read lz4 file, decompress and load arrays
t1 = perf_counter()
arr_dict = decompress_and_convert(output_file, out_format = 'npz')
t2 = perf_counter()
print('Time to read npz file using lz4: ', t2-t1)

Time to compress npz file using lz4:  0.14694500000001653
Input file size: 50.06804656982422 MB
Output file size: 50.0709924697876 MB
Compression ratio: 1.00x
Time to read npz file using lz4:  0.05856810000000223
